#BERTopic on Quran Urdu Translations
In this notebook I have implemented Topic Modeling on Urdu translations based topic modelling technique BERTopic.

Shaista Zulfiqar

## Mounting Google Drive
If the dataset is on Google Drive then you have to mount over google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




#Installing required dependencies
**One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.**

In [ ]:
!pip install bertopic
!pip install -U sentence-transformers
!pip install urduhack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB

In [ ]:
!pip install --upgrade keras tensorflow-addons


# Importing required dependencies
We will import numpy, pandas and re, bertopic, gensim library for now. other libraries will be imported in the notebook later.

Pandas will be used to create a Dataframe and handle the csv file. Numpy will be used for the faster computation of arrays to save time. re library will be used for the cleaning of data. gensim library is used to get coherence score and train LDA. bertopic is used to train bertopic on our Quran-UTM dataset with using pretrained language models Multilingial MiniLM

In [ ]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from urduhack.normalization import remove_diacritics #Rerun this cell if you get any error
from gensim.models import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
#optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

##DataFrame




In [ ]:

junapd = pd.read_table("path of dataset", header=None, encoding='utf-8')

print(junapd.head(5))

stopspd=pd.read_csv('/content/drive/MyDrive/stopwords.txt',names=['List'])

stopspd

                                                   0
0  1|1|شروع الله کا نام لے کر جو بڑا مہربان نہایت...
1  1|2|سب طرح کی تعریف خدا ہی کو (سزاوار) ہے جو ت...
2                      1|3|بڑا مہربان نہایت رحم والا
3                            1|4|انصاف کے دن کا حاکم
4  1|5|(اے پروردگار) ہم تیری ہی عبادت کرتے ہیں او...


,List
0,کی
1,ہیں
2,ہے
3,رہا
4,رہی
...,...
396,گئی
397,ہونے
398,وجہ
399,ہوگیا


## Cleaning of Data
After collecting the eight Quran Urdu translations, we observed these translation had some irrelevant information, such as metadata, punctuation, and diacritics. So we cleaned the translations for topic modeling

Stopwords are common words that are often filtered out during text processing in natural language processing (NLP) tasks. These words are considered to have little or no value in conveying the actual meaning of the text. We take list of 401 stopwords for topic modelling. Stopwords are removed in post preprocessing phase

In [ ]:
import re
#Remove Urdu Punctuation
def remove_urdu_punctuation(text):
    pattern = r'[؛؟،٫٬‘’“”«»!"٪&\'\*\+,-./:;<=>؟@^_`()[]{|}~]'

    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text


In [ ]:
# Remove trailing metadata lines
index_to_drop = junapd[junapd[0].str.startswith("#")].index
junapd.drop(index_to_drop, inplace=True)

# Remove verse reference from every line
junapd[0] = junapd[0].str.replace(r'\d+\|\d+\|', '', regex=True)

# Remove diacritics
junapd[0] = junapd[0].apply(remove_diacritics)

# Remove punctuation
junapd[0] = junapd[0].apply(remove_urdu_punctuation)

# Reset index
junapd.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(junapd.head())



Cleaned DataFrame:
                                                   0
0  شروع الله کا نام لے کر جو بڑا مہربان نہایت رحم...
1  سب طرح کی تعریف خدا ہی کو (سزاوار) ہے جو تمام ...
2                          بڑا مہربان نہایت رحم والا
3                                انصاف کے دن کا حاکم
4  (اے پروردگار) ہم تیری ہی عبادت کرتے ہیں اور تج...


In [ ]:
def remove_nonbreaking_space(text):
    return re.sub(r'\xa0', ' ', text)

junapd[0] = junapd[0].apply(remove_nonbreaking_space)

In [ ]:
# Save the cleaned DataFrame to a text file
file_path = 'cleaned_Jalandhry_text.txt'
junapd[0].to_csv(file_path, sep='\n', index=False, header=False, encoding='utf-8')

print(f'Text has been saved to {file_path}')

Text has been saved to cleaned_Jalandhry_text.txt


In [ ]:
# Convert the DataFrame to a list of strings
data = junapd[0].tolist()

In [ ]:
print(len(data))

6236


In [ ]:
print(data[56:90])

['اور جب ہم نے تمہارے لیے دریا کو پھاڑ دیا تم کو نجات دی اور فرعون کی قوم کو غرق کر دیا اور تم دیکھ ہی تو رہے تھے', 'اور جب ہم نے موسی سے چالیس رات کا وعدہ کیا تو تم نے ان کے پیچھے بچھڑے کو (معبود) مقرر کر لیا اور تم ظلم کر رہے تھے', 'پھر اس کے بعد ہم نے تم کو معاف کر دیا، تاکہ تم شکر کرو', 'اور جب ہم نے موسی کو کتاب اور معجزے عنایت کئے، تاکہ تم ہدایت حاصل کرو', 'اور جب موسی نے اپنی قوم کے لوگوں سے کہا کہ بھائیو، تم نے بچھڑے کو (معبود) ٹھہرانے میں (بڑا) ظلم کیا ہے، تو اپنے پیدا کرنے والے کے آگے توبہ کرو اور اپنے تئیں ہلاک کر ڈالو۔ تمہارے خالق کے نزدیک تمہارے حق میں یہی بہتر ہے۔ پھر اس نے تمہارا قصور معاف کر دیا۔ وہ بے شک معاف کرنے والا (اور) صاحب رحم ہے', 'اور جب تم نے (موسی) سے کہا کہ موسی، جب تک ہم خدا کو سامنے نہ دیکھ لیں گے، تم پر ایمان نہیں لائیں گے، تو تم کو بجلی نے آ گھیرا اور تم دیکھ رہے تھے', 'پھر موت آ جانے کے بعد ہم نے تم کو ازسر نو زندہ کر دیا، تاکہ احسان مانو', 'اور بادل کا تم پر سایہ کئے رکھا اور (تمہارے لیے) من و سلوی اتارتے رہے کہ جو پاکیزہ چیزیں ہم نے تم کو عطا فرمائی 

# BERTopic Training
The default  bertopic embedding model is paraphrase-multilingual-MiniLM-L12-v2 when selecting language="multilingual". We take MiniLM Model from [sentence-tranformer](https://www.sbert.net/docs/pretrained_models.html) and create custom document embedding and passed it to the bertopic model for training.

In [ ]:
#create custom embedding
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(data, show_progress_bar=True)
print(embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/195 [00:00<?, ?it/s]

[[ 0.02631805  0.42147276 -0.14640112 ... -0.1071908   0.31291097
   0.08368939]
 [-0.0128808   0.47506043 -0.07064087 ... -0.10684063  0.2616918
   0.08328619]
 [ 0.02005923  0.26257578 -0.03005545 ... -0.20040497  0.23603514
   0.05042844]
 ...
 [ 0.0855466   0.27319062 -0.17135492 ... -0.05320187  0.33026075
   0.09511978]
 [-0.03382144  0.00611624 -0.11566864 ... -0.04479951  0.1560542
   0.0047373 ]
 [-0.05599292  0.21802457 -0.05016701 ...  0.00659058  0.21080826
  -0.02342556]]


In [ ]:
stop_words_list = stopspd['List'].tolist()

In [ ]:
#pass vectorizer_model to bertopic for stopwords removal
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words= stop_words_list)


In [ ]:
#UMAP for dimention reduction
from umap import UMAP
dim_model = UMAP(n_components=4, random_state=42)

In [ ]:
# # #KMeans used for clustering
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=5, random_state=42)

In [ ]:
np.random.seed(42)

In [ ]:
topic_model = BERTopic(language="urdu", low_memory=True ,calculate_probabilities=True,  top_n_words=10,hdbscan_model=cluster_model, umap_model=dim_model, verbose=True, vectorizer_model=vectorizer_model)

In [ ]:
#Fit documents in bertopic
topics, probs = topic_model.fit_transform(data,embeddings)

2024-06-09 14:31:11,406 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-09 14:31:59,270 - BERTopic - Dimensionality - Completed ✓
2024-06-09 14:31:59,273 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-09 14:31:59,456 - BERTopic - Cluster - Completed ✓
2024-06-09 14:31:59,468 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-09 14:31:59,677 - BERTopic - Representation - Completed ✓


In [ ]:
print(probs)

None


In [ ]:
#topics that assign to each document
print(topics)

[1, 1, 1, 0, 1, 0, 0, 0, 3, 2, 3, 1, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 1, 2, 1, 1, 1, 1, 1, 2, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 2, 1, 3, 0, 1, 3, 1, 1, 1, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 2, 2, 3, 1, 2, 3, 1, 0, 0, 3, 1, 1, 1, 3, 0, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 3, 0, 2, 2, 3, 2, 2, 2, 3, 3, 3, 3, 2, 2, 3, 2, 2, 2, 3, 3, 3, 0, 1, 2, 1, 3, 2, 2, 3, 1, 3, 3, 1, 1, 1, 2, 0, 3, 3, 3, 0, 3, 3, 1, 3, 1, 3, 3, 3, 3, 1, 0, 3, 3, 2, 1, 1, 3, 0, 3, 3, 3, 2, 3, 3, 1, 1, 3, 3, 0, 3, 2, 2, 2, 1, 3, 2, 1, 2, 0, 1, 1, 2, 2, 2, 2, 2, 3, 2, 3, 1, 2, 2, 2, 1, 1, 0, 2, 2, 1, 0, 0, 3, 3, 2, 0, 1, 2, 3, 1, 2, 2, 1, 1, 3, 1, 1, 3, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 3, 2, 3, 3, 1, 3, 3, 2, 3, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 1, 2, 3, 1, 3, 3, 3, 1, 1, 2, 3, 3, 0, 1, 3, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 0, 1, 3, 3, 1, 1, 3, 1, 1, 2, 3, 3, 3, 1, 2, 1, 2, 1, 3, 3, 2, 2, 3, 2, 0, 1, 1, 1, 3, 1, 3, 2, 3, 2, 1, 1, 3, 1, 3, 1, 

In [ ]:
topic_model.get_topic_freq()

,Topic,Count
1,0,2208
0,1,1635
3,2,1259
2,3,1103
4,4,31


In [ ]:
document_topics = topic_model.get_topics()

In [ ]:
#topics with score
print(document_topics)

{0: [('دن', 0.03324379729471399), ('لوگ', 0.028081362683904515), ('لوگوں', 0.02620835461271456), ('عذاب', 0.026176176038982434), ('پیدا', 0.02429330984652383), ('تمہارے', 0.024038660511372933), ('پاس', 0.023538745203991542), ('دو', 0.022635798614829523), ('وقت', 0.02249725297878098), ('زمین', 0.021726757815702505)], 1: [('خدا', 0.10031866286668599), ('پروردگار', 0.08127309056607034), ('والا', 0.04557600385055498), ('تمہارے', 0.042349082718886794), ('زمین', 0.037184784128577876), ('پیدا', 0.027556066467463145), ('دو', 0.027269108219531723), ('تمہارا', 0.027176742948947664), ('دن', 0.025984122246642417), ('آسمانوں', 0.02595980319565625)], 2: [('خدا', 0.10129677240892679), ('لوگ', 0.06109801033356642), ('لوگوں', 0.045535706293579496), ('ایمان', 0.037877468990896605), ('عذاب', 0.03721948664123689), ('دوزخ', 0.02975599381896181), ('کوئی', 0.026152084798071306), ('والا', 0.025468855008388615), ('سوا', 0.024085773353254902), ('تمہارے', 0.022678592243866834)], 3: [('خدا', 0.07121584570677043),

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2208,0_دن_لوگ_لوگوں_عذاب,"[دن, لوگ, لوگوں, عذاب, پیدا, تمہارے, پاس, دو, ...",[اس نے کہا کہ مجھے اس دن تک مہلت عطا فرما جس د...
1,1,1635,1_خدا_پروردگار_والا_تمہارے,"[خدا, پروردگار, والا, تمہارے, زمین, پیدا, دو, ...",[اور آسمانوں اور زمین کی بادشاہی خدا ہی کی ہے۔...
2,2,1259,2_خدا_لوگ_لوگوں_ایمان,"[خدا, لوگ, لوگوں, ایمان, عذاب, دوزخ, کوئی, وال...",[اور بعض لوگ ایسے ہیں جو کہتے ہیں کہ ہم خدا پر...
3,3,1103,3_خدا_پیغمبر_اے_موسی,"[خدا, پیغمبر, اے, موسی, لوگوں, کتاب, پاس, تمہا...",[اور کافر لوگ کہتے ہیں کہ اس (پیغمبر) پر اس کے...
4,4,31,4_کون_جھٹلاؤ_نعمت_سی,"[کون, جھٹلاؤ, نعمت, سی, پروردگار, وانس, گروہ, ...",[تو تم اپنے پروردگار کی کون کون سی نعمت کو جھٹ...


In [ ]:
topic_distr, _ = topic_model.approximate_distribution(data, window=3, min_similarity=0.01)

100%|██████████| 7/7 [00:02<00:00,  3.08it/s]


In [ ]:
print(topic_distr)

[[0.14247755 0.42118534 0.20231299 0.23402412 0.        ]
 [0.04367751 0.42225138 0.23573735 0.21034182 0.08799194]
 [0.1777406  0.44601366 0.18834535 0.18790039 0.        ]
 ...
 [0.09505752 0.3230373  0.35667228 0.22523291 0.        ]
 [0.21226674 0.17127737 0.36274048 0.25371541 0.        ]
 [0.         0.         1.         0.         0.        ]]


In [ ]:
topic_model.visualize_distribution(topic_distr[0], width=600,height=600, title="Topic Probability Distributio")

# Evaluation
we used three evaluation metrics to compare the results.

1. The coherence score is used to capture the degree of similarity between the words within each topic, with higher scores indicating more coherent topics. We used two coherence metrics NPMI and Cv Score.
2. IRBO measures are used to assess how different and distinct the topics are in a topic model.


### Coherence Score
To evaluate the model topics coherence we use [Gensim](https://radimrehurek.com/gensim/models/coherencemodel.html) library

In [ ]:
texts = [[word for word in str(document).split() if word not in stop_words_list] for document in data] #if word not in stop_words_list
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
topics_bert=[]
for i in topic_model.get_topics():
  row=[]
  topic= topic_model.get_topic(i)
  for word in topic:
     row.append(word[0])
  topics_bert.append(row)

In [ ]:
print(topics_bert)

[['دن', 'لوگ', 'لوگوں', 'عذاب', 'پیدا', 'تمہارے', 'پاس', 'دو', 'وقت', 'زمین'], ['خدا', 'پروردگار', 'والا', 'تمہارے', 'زمین', 'پیدا', 'دو', 'تمہارا', 'دن', 'آسمانوں'], ['خدا', 'لوگ', 'لوگوں', 'ایمان', 'عذاب', 'دوزخ', 'کوئی', 'والا', 'سوا', 'تمہارے'], ['خدا', 'پیغمبر', 'اے', 'موسی', 'لوگوں', 'کتاب', 'پاس', 'تمہارے', 'قرآن', 'لوگ'], ['کون', 'جھٹلاؤ', 'نعمت', 'سی', 'پروردگار', 'وانس', 'گروہ', 'اے', '', '']]


In [ ]:
# compute Coherence Score CV

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_v')
coherence = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence)


CV Score:  0.51


In [ ]:
# compute Coherence Score NPMI

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_npmi')
coherence = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence)


NPMI Score:  0.02


**Diversity Score**

upload rbo.py file before importing

In [ ]:
import itertools
from rbo import rbo
import numpy as np

class InvertedRBO:
    def __init__(self):
        pass

    def irbo(self, topics, topk=10, weight=0.9):
        """
        Calculate inverted Rank Biased Overlap (RBO) as a measure of topic diversity from a list of lists of words.

        :param topics: A list of lists of words representing different topics.
        :param topk: The number of top words on which RBO will be computed.
        :param weight: Weight of each agreement at depth d: p**(d-1). When set to 1.0, there is no weight,
                       and the RBO returns to average overlap.
        :return: The inverted RBO topic diversity score.
        """
        if topk <= 0:
            raise ValueError("topk must be a positive integer.")

        num_topics = len(topics)
        if num_topics == 0:
            raise ValueError("topics list cannot be empty.")

        if topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')

        collect = []
        for list1, list2 in itertools.combinations(topics, 2):
            rbo_val = rbo(list1[:topk], list2[:topk], p=weight)[2]
            collect.append(rbo_val)

        Irbo_score = 1 - np.mean(collect)
        return Irbo_score

In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= round(inverted_rbo_calculator.irbo(topics_bert, topk=10, weight=0.9),2)
print("Inverted RBO Score:", IRBO)

Inverted RBO Score: 0.79


# Visualize Topics

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(n_words=10,width=220, height=270, title="Topic Word Scores")

# Model serialization

In [ ]:
# Save model
topic_model.save("my_model")

2024-06-09 14:34:19,506 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
loaded_topic_model = topic_model.load("my_model")
loaded_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,2208,0_دن_لوگ_لوگوں_عذاب,"[دن, لوگ, لوگوں, عذاب, پیدا, تمہارے, پاس, دو, ...",[اس نے کہا کہ مجھے اس دن تک مہلت عطا فرما جس د...
1,1,1635,1_خدا_پروردگار_والا_تمہارے,"[خدا, پروردگار, والا, تمہارے, زمین, پیدا, دو, ...",[اور آسمانوں اور زمین کی بادشاہی خدا ہی کی ہے۔...
2,2,1259,2_خدا_لوگ_لوگوں_ایمان,"[خدا, لوگ, لوگوں, ایمان, عذاب, دوزخ, کوئی, وال...",[اور بعض لوگ ایسے ہیں جو کہتے ہیں کہ ہم خدا پر...
3,3,1103,3_خدا_پیغمبر_اے_موسی,"[خدا, پیغمبر, اے, موسی, لوگوں, کتاب, پاس, تمہا...",[اور کافر لوگ کہتے ہیں کہ اس (پیغمبر) پر اس کے...
4,4,31,4_کون_جھٹلاؤ_نعمت_سی,"[کون, جھٹلاؤ, نعمت, سی, پروردگار, وانس, گروہ, ...",[تو تم اپنے پروردگار کی کون کون سی نعمت کو جھٹ...


# LDA

We use the [ parallelized Latent Dirichlet Allocation (LDA)](https://radimrehurek.com/gensim/models/ldamulticore.html) from Gensim.

Note: for LDA you have to define topics number in advance.

In [ ]:
n_topics=5
lda = LdaMulticore(corpus, id2word=id2word, random_state=42, num_topics=5)
topics = lda.show_topics(num_topics=5, formatted=False)

#Extract the words from the topics
topics_list = []
for _, topic_words in topics:
    words = [word for word, _ in topic_words]
    topics_list.append(words)

print(topics_list)

In [ ]:
#CV Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_lda = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_lda)


CV Score:  0.41


In [ ]:
#NPMI Score
cm = CoherenceModel(topics=topics_list, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_lda = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_lda)


NPMI Score:  -0.05


In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_LDA = round(inverted_rbo_calculator.irbo(topics_list, topk=10, weight=0.9),2)
print('\nIRBO Score: ',IRBO_LDA)


IRBO Score:  0.54


#NMF
We use Gensim library for implementation of NMF

Note: for NMF you have to define topics number in advance.

In [ ]:
#Using Gensim
from gensim import corpora, models


# Train NMF model
num_topics = 5  # Define the number of topics
nmf_model = models.Nmf(corpus, num_topics=num_topics, id2word=id2word, random_state=42)

# Extract topics
topics = []
for topic_id in range(num_topics):
    topic_words = nmf_model.show_topic(topic_id, topn=10)
    topic_words = [word for word, _ in topic_words]
    topics.append(topic_words)

# Print topics in the desired format
print(topics)

In [ ]:
#Calculate CV Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence_nmf = round(cm.get_coherence(),2)
print('\nCV Score: ', coherence_nmf)

In [ ]:
#Calculate NPMI Score
cm = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_npmi')
coherence_nmf = round(cm.get_coherence(),2)
print('\nNPMI Score: ', coherence_nmf)

In [ ]:
# Calculate IRBO Score
inverted_rbo_calculator = InvertedRBO()
IRBO_NMF = round(inverted_rbo_calculator.irbo(topics, topk=10, weight=0.9),2)c
print('\nIRBO Score: ',IRBO_NMF)